# OpenAI APIs Assignment: Prompt Engineering & Agents

## Assignment Overview
This assignment focuses on two main scenarios:
* Tool Use with Wikipedia: Implementing function calling with Wikipedia integration
** AI Agent Development: Creating specialized agents using OpenAI Agents SDK

## Assignment Structure
Part 1: Tool Use with Wikipedia Integration 
Objective: Create a research assistant that can search Wikipedia and provide comprehensive information using OpenAI's function calling capabilities.

## Requirements:
* Wikipedia Search Function: Implement a function to search Wikipedia articles
* Content Extraction: Extract and summarize article content
* Function Calling: Use OpenAI's function calling to integrate Wikipedia data
* Structured Output: Return information in a structured format
* Error Handling: Handle cases where articles don't exist or are ambiguous

## Key Implementation Tasks:

```# Required libraries
import wikipedia
import openai
from dotenv import load_dotenv

# Function 1: Wikipedia Search
def search_wikipedia(query):
    """Search Wikipedia for articles related to the query"""
    # Students implement Wikipedia search using wikipediaapi
    # Return structured results with titles and descriptions

# Function 2: Content Extraction  
def get_wikipedia_content(title):
    """Get full content of a Wikipedia article"""
    # Students implement content extraction
    # Handle missing articles and disambiguation

# Function 3: OpenAI Function Calling
def handle_wikipedia_research(query):
    """Complete function calling workflow"""
    # Students implement the full workflow:
    # 1. Send query to OpenAI with function definitions
    # 2. Handle function calls (search_wikipedia, get_wikipedia_content)
    # 3. Execute requested functions
    # 4. Send results back to OpenAI for final response
```

## You can use one of these test cases to show:
* "What is artificial intelligence?"
* "Tell me about the history of Singapore"
* "Explain quantum computing"


In [2]:
#Wikipedia Search Function
import wikipedia

def get_article(search_term):
    results = wikipedia.search(search_term)
    first_result = results[0]
    page = wikipedia.page(first_result, auto_suggest=False)
    return page.content

article = get_article("What is artificial intelligence?")
print(article[:1000]) # article is very long, so let's just print a preview

Artificial intelligence (AI) is the capability of computational systems to perform tasks typically associated with human intelligence, such as learning, reasoning, problem-solving, perception, and decision-making. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and use learning and intelligence to take actions that maximize their chances of achieving defined goals.
High-profile applications of AI include advanced web search engines (e.g., Google Search); recommendation systems (used by YouTube, Amazon, and Netflix); virtual assistants (e.g., Google Assistant, Siri, and Alexa); autonomous vehicles (e.g., Waymo); generative and creative tools (e.g., language models and AI art); and superhuman play and analysis in strategy games (e.g., chess and Go). However, many AI applications are not perceived as AI: "A lot of cutting edge AI has filtered into general applications, often without being calle

In [ ]:
#Assignment 1 - Your Answer 